<a href="https://colab.research.google.com/github/imabari/toyama/blob/master/toyama_kanja_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jaconv

  Created wheel for jaconv: filename=jaconv-0.2.4-cp36-none-any.whl size=12285 sha256=660f86377714e171f65a44e309d191b9b67f14ddf536bd05001da29b79050fe3
  Stored in directory: /root/.cache/pip/wheels/e1/46/f7/85a7f89bd3263423c8530dfed16083f9a142cc0fc78c81ff32
Successfully built jaconv


In [0]:
import jaconv

In [0]:
import datetime

In [0]:
import requests
from bs4 import BeautifulSoup

In [0]:
url = "http://www.pref.toyama.jp/cms_sec/1205/kj00021798.html"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko",
}

In [0]:
# 和暦から西暦のdateに変換
def wareki2date(s):

    m = re.match("(昭和|平成|令和)(\d{1,2})年(\d{1,2})月(\d{1,2})日", s)

    year = int(m.group(2))
    month = int(m.group(3))
    day = int(m.group(4))

    if m.group(1) == "昭和":
        year += 1925
    elif m.group(1) == "平成":
        year += 1988
    elif m.group(1) == "令和":
        year += 2018

    result = datetime.date(year, month, day)

    return result

In [0]:
r = requests.get(url, headers=headers)

r.raise_for_status()

soup = BeautifulSoup(r.content, "html5lib")

In [0]:
import re

In [0]:
result = []

h4 = soup.find("h4")

title = jaconv.z2h(h4.get_text(), digit=True, ascii=True)

dt = wareki2date(title)

s = jaconv.z2h(
    h4.find_next_sibling("p").get_text("\n", strip=True),
    kana=False,
    digit=True,
    ascii=False,
).strip("◎")

for i in s.split("◎"):

    data = [dt]

    tmp = i.splitlines()

    m = re.match("(\d{1,3})例目（(.+)）", tmp[0])
    data.extend(list(m.groups()))

    data.extend([i.split()[1] for i in tmp[1:]])

    result.append(data)

In [0]:
import pandas as pd

In [0]:
df = pd.DataFrame(result, columns=["公表年月日", "No", "症状", "年代", "性別", "居住地"])

In [0]:
df["No"] = df["No"].astype(int)

In [0]:
df["症状"] = df["症状"].replace({"患者":"軽症・中等症", "無症状病原体保有者":"無症状"})

In [0]:
df["年代"] = df["年代"].str.replace("歳", "")

In [0]:
df["市区町村名"] = df["居住地"].apply(lambda x: "富山市" if "富山市" in x else "")

In [0]:
df["状態"] = "入院中"

In [0]:
df["発症日"] = ""
df["職業"] = ""
df["渡航歴の有無"] = "x"
df["備考"] = ""

In [0]:
df.set_index("No", inplace=True)

In [0]:
df.sort_index(inplace=True)

In [0]:
df_csv = df.loc[
    :, ["市区町村名", "公表年月日", "発症日", "居住地", "年代", "性別", "職業", "状態", "症状", "渡航歴の有無", "備考"]
]

In [25]:
df_csv

,市区町村名,公表年月日,発症日,居住地,年代,性別,職業,状態,症状,渡航歴の有無,備考
No,,,,,,,,,,,
6,富山市,2020-04-02,,富山市保健所管内（富山市）,50代,女性,,入院中,軽症・中等症,x,
7,富山市,2020-04-02,,富山市保健所管内（富山市）,20代,男性,,入院中,無症状,x,
8,,2020-04-02,,高岡厚生センター管内（高岡市、射水市、氷見市）,50代,女性,,入院中,軽症・中等症,x,


In [0]:
df_csv.to_csv(f"{dt}_kanja.csv", encoding="utf_8_sig")

In [0]:
from google.colab import files

In [0]:
files.download(f"{dt}_kanja.csv")